# Análisis Estadístico Multivariado
# 29 de Noviembre de 2025
# Proyecto final
# Acoyani Garrido Sandoval

## 1. Descripción

Consiste en determinar la relación entre una variable dependiente y una o más variables independientes a través de regresión logística múltiple. Se usará para ello un conjunto de datos de patrones de órdenes de comida rápida en diferentes ciudades de India, desarrollado por Prince Rajak y disponible en [Kaggle](https://www.kaggle.com/datasets/prince7489/fast-food-ordering-pattern-dataset).



In [ ]:
import pandas
import numpy
import matplotlib.pyplot as pyplot
import json
import seaborn

# Boilerplate: necesitamos un JSON encoder capaz de manejar números de Numpy
class SerializadorDeJSONParaLaClaseDeTemores(json.JSONEncoder):
   def default(self, objeto):
      # numpy.int64: lo convertimos a un número estándar de Python para que el serializador estándar
      # pueda manejarlo
      if isinstance(objeto, numpy.int64):
         return objeto.item()
      # Cualquier otro tipo: usamos el serializador estándar
      else:
         return json.JSONEncoder.default(self, objeto)


# Tomamos la primera columna como índice, porque es un número de ID
df_pedidos = pandas.read_csv("fast_food_ordering_dataset.csv", index_col=0)

## 2. Análisis descriptivo

Comenzamos por desarrollar un análisis descriptivo de las variables del conjunto de datos.

Cada registro de nuestro conjunto de datos es un pedido individual de comida rápida, cuyas variables son:
- **order_id:** Número de pedido
- **order_time:** Fecha y hora del pedido en tiempo de India
- **city:** Ciudad donde fue hecho el pedido
- **cuisine_type:** Estilo de cocina
- **order_value:** Precio del pedido en rupias
- **delivery_time_minutes:** Tiempo del pedido en minutos
- **items_count:** Cuántos elementos tiene el pedido
- **payment_method:** Método de pago:
   - **UPI:** *Unified Payments Interface,* un sistema de pagos electrónicos operado por el banco nacional de India
   - **Wallet:** pagos hechos mediante sistemas orientados a dispositivos móviles, tales como Apple y Google Pay.
   - **Credit Card:** tarjeta de crédito
   - **Debit Card:** tarjeta de débito
   - **Cash:** efectivo

El análisis que haremos incluirá:

1. **Para las variables de razón:** Medidas de tendencia central y de dispersión: moda, media, mediana, rango, desviación estándar, rango intercuartílico, y diagrama de caja.
   - Minutos de entrega
   - Cantidad de elementos
   - Precio
1. **Para la variable de intervalo:** Distribución (histograma), moda y mediana
   - Hora del pedido
1. **Para las variables nominales:** Conteo en gráfica, y porcentaje
   - Ciudad
   - Estilo de cocina
   - Método de pago


In [10]:
variables_razon = [ "delivery_time_minutes", "items_count", "order_value" ]
variables_intervalo = [ "order_time" ]
variables_nominales = [ "city", "cuisine_type", "payment_method" ]

# Medidas de las variables de razón
variables_razon_medidas = {}
for una_variable in variables_razon:
   medidas = \
   {
      "promedio": df_pedidos[una_variable].mean(),
      "mediana": df_pedidos[una_variable].median(),
      "moda": df_pedidos[una_variable].mode()[0],
      "varianza": df_pedidos[una_variable].var(),
      "desvstd": df_pedidos[una_variable].std(),
      "rango": df_pedidos[una_variable].max() - df_pedidos[una_variable].min(),
      "RI": df_pedidos[una_variable].quantile(0.75) - df_pedidos[una_variable].quantile(0.25)
   }
   variables_razon_medidas.update({ una_variable: medidas })

print(f"Medidas de las variables de razón: {json.dumps(variables_razon_medidas, indent=3, cls=SerializadorDeJSONParaLaClaseDeTemores)}")


Medidas de las variables de razón: {
   "delivery_time_minutes": {
      "promedio": 44.696,
      "mediana": 44.0,
      "moda": 43,
      "varianza": 299.28214829659316,
      "desvstd": 17.29977307066752,
      "rango": 59,
      "RI": 30.25
   },
   "items_count": {
      "promedio": 4.104,
      "mediana": 4.0,
      "moda": 5,
      "varianza": 3.8208256513026053,
      "desvstd": 1.9546932371353325,
      "rango": 6,
      "RI": 4.0
   },
   "order_value": {
      "promedio": 630.14666,
      "mediana": 619.0,
      "moda": 105.1,
      "varianza": 98622.8453276998,
      "desvstd": 314.04274442772885,
      "rango": 1094.5900000000001,
      "RI": 546.9575
   }
}


In [ ]:
# Medidas de la variable de intervalo: moda y mediana
# Pero antes, tengo que convertir esa columna a timestamps de Numpy
# Tratándose de la hora del día, aquí es sensato sacar la distribución en cubetas de 24 horas


variables_intervalo_medidas = {}

